In [2]:
import keras

ImportError: No module named keras

In [ ]:
model <- keras_model_sequential()

model %>%
  # Begin with 2D convolutional LSTM layer
  layer_conv_lstm_2d(
    input_shape = list(24,40,40,2),
    filters = 256, kernel_size = c(3,3),
    padding = "same",
    return_sequences = TRUE,
    data_format = "channels_last"
  ) %>% 
  # Add 3x hidden 2D convolutions LSTM layers
  layer_conv_lstm_2d(
    filters = 128, kernel_size = c(3,3),
    padding = "same", return_sequences = TRUE
  ) %>%
  layer_conv_lstm_2d(
    filters = 128, kernel_size = c(3,3),
    padding = "same", return_sequences = TRUE
  ) %>%
  layer_conv_lstm_2d(
    filters = 1, kernel_size = c(3,3),
    padding = "same", return_sequences = TRUE
  )

# Prepare model for training
model %>% compile(
  loss = "mse",
  metrics = "mae",
  optimizer = "adam"
)

history <- model %>% fit(
  x_train,
  y_train,
  batch_size = 15,
  epochs = 40,
  validation_data = list(x_validation, y_validation)
)

plot(history)

